<a href="https://colab.research.google.com/github/simonebugo/Big_Data/blob/main/Esercizio_in_aula6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Il dataset “heart.csv” è stato preso da kaggle (https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction)
e contiene dati relativi ad alcuni pazienti:

● Age: età del paziente [anni]

● Sex: sesso del paziente [M: maschio, F: femmina]

● ChestPainType: tipo di dolore toracico [TA: Angina tipica, ATA: Angina atipica, NAP: Dolore non anginoso, ASY: Asintomatico]

● RestingBP: pressione sanguigna a riposo [mm Hg]

● Cholesterol: colesterolo sierico [mm/dl]

● FastingBS: zucchero nel sangue a digiuno [1: se FastingBS > 120 mg/dl, 0: altrimenti]

● RestingECG: elettrocardiogramma a riposo [Normale, ST: anomalia dell'onda T-ST, LVH: ipertrofia ventricolare sinistra]

● MaxHR: frequenza cardiaca massima raggiunta [valore numerico tra 60 e 202]

● ExerciseAngina: angina indotta da esercizio [SÌ, NO]

● Oldpeak: valore numerico misurato in depressione

● ST_Slope: pendenza del picco [Up: in salita, Flat: piatto, Down: in discesa]

● HeartDisease: classe da predire [1: cardiopatia, 0: normale]

In [4]:
import pandas as pd
import numpy as np

1. A partire dal dataset originale, eliminare eventuali attributi inutili (giustificare la scelta), eliminare eventuali istanze che
contengono valori nulli, trasformare opportunamente valori categorici e dividere il dataset in modo che 3/4 degli elementi
siano contenuti in un nuovo dataset “train” e 1/4 nel dataset “test” preservando le proporzioni delle classi nel target.

In [6]:
df = pd.read_csv("heart.csv")
df.head() #il target è heartdease

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [7]:
df.shape

(918, 12)

In [8]:
df.isnull().sum() #vedo se ci sono valori nulli

,0
Age,0
Sex,0
ChestPainType,0
RestingBP,0
Cholesterol,0
FastingBS,0
RestingECG,0
MaxHR,0
ExerciseAngina,0
Oldpeak,0


In [9]:
df["HeartDisease"].value_counts() #vedo se target bilanciato o meno. conto numero di sample per ogni classe colonna target

,count
HeartDisease,
1,508
0,410


In [10]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [11]:
df.dtypes #vedo che ci sono feature categoriche che andranno trasformate

,0
Age,int64
Sex,object
ChestPainType,object
RestingBP,int64
Cholesterol,int64
FastingBS,int64
RestingECG,object
MaxHR,int64
ExerciseAngina,object
Oldpeak,float64


In [12]:
#potrei costuire dei dizionari per fare il replace oppure il modo più forbo è usare il lable encoder
#seleziono tutte le colonne di tipo obj e le trasformo con un fit trasform. converto ogni colonna non numerica in una colonna numerica
df = df.dropna()

le = LabelEncoder()
for column in df:
  if df[column].dtype == object:
    df[column] = le.fit_transform(df[column]) #trasformo le colonne con il fit trasform

#con funzione train test split divido il dataset nella porzione per train e per il test. devo escludere il target per passargli le feature
#uso funzione stratify per dividere in modo equo le classi. rispetto stessa proporzione di 0 e 1 nel target nella divisone train e set
#il random state serve per ottenere sempre lo stesso risultato
X_train, X_test, Y_train, Y_test = train_test_split(df.drop('HeartDisease', axis=1), df['HeartDisease'], test_size=0.25, stratify=df['HeartDisease'], random_state=42)

2. Utilizzando il dataset train, allenare il modello DecisionTree e valutare l’accuratezza ottenuta sia sul dataset train sia sul
dataset test. Confrontare i risultati ottenuti con quelli ottenuti con una predizione basata sul modello
KNeighborsClassifier e con la predizione effettuata da un dummy classifier a scelta.

In [16]:
#alleno 3 diversi classificatori sul training e poi testarli sul test
models = {'Dummy': DummyClassifier(strategy="most_frequent"), #farà una predizione che consiste nel predirre la classe più frequente
          'Decision Tree': DecisionTreeClassifier(),
          'KNN': KNeighborsClassifier()}
#costruisco funzione che fa ad ogni volta trainingdel modello diverso, prediction sul training e prediction sul test
#calcola poi le diverse metriche che vogliamo valutare, ovvero accuracy e f1.
def test_model(name, model, traindata, testdata, trainlabels, testlabels):
  model.fit(traindata, trainlabels)
  trainpred = model.predict(traindata)
  testpred = model.predict(testdata)
  print(f" ===== {name} =====")
  print(f"Accuracy on train {accuracy_score(trainlabels, trainpred)}")
  print(f"F1score on train {f1_score(trainlabels, trainpred)}")
  print(f"Accuracy on test {accuracy_score(testlabels, testpred)}")
  print(f"F1score on test {f1_score(testlabels, testpred)}")
  print("Confusion matrix on test")
  print(confusion_matrix(testlabels, testpred))

for name, model in models.items():
  test_model(name, model, X_train, X_test, Y_train, Y_test)

 ===== Dummy =====
Accuracy on train 0.5537790697674418
F1score on train 0.7128157156220767
Accuracy on test 0.5521739130434783
F1score on test 0.711484593837535
Confusion matrix on test
[[  0 103]
 [  0 127]]
 ===== Decision Tree =====
Accuracy on train 1.0
F1score on train 1.0
Accuracy on test 0.8043478260869565
F1score on test 0.8163265306122449
Confusion matrix on test
[[ 85  18]
 [ 27 100]]
 ===== KNN =====
Accuracy on train 0.7790697674418605
F1score on train 0.8025974025974026
Accuracy on test 0.691304347826087
F1score on test 0.7215686274509804
Confusion matrix on test
[[67 36]
 [35 92]]


3. Confrontare l’accuratezza ottenuta nel punto precedente con l’accuratezza che si ottiene con una 10 Fold cross
validation.

In [13]:
from sklearn.model_selection import cross_val_score
#do tutto dataset in pasto alla cross validation e vedo quali sono le performanc emedie sui vari split del dataset

In [17]:
Y = df["HeartDisease"] #solo il target
X = df.drop("HeartDisease", axis=1) #tutto dataset meno il target

In [18]:
for name, model in models.items():
  # Uso X e Y interi perchè farà lui la divisione in test e train
  result = cross_val_score(model, X, Y, cv=10)
  print(f" ===== {name} =====")
  print(f"Media {result.mean()}")
  print(result)
  #l'accuracy ottenuta non dipedne dallo split specifico ma è una media di 10 split

 ===== Dummy =====
Media 0.553368370759675
[0.55434783 0.55434783 0.55434783 0.55434783 0.55434783 0.55434783
 0.55434783 0.55434783 0.54945055 0.54945055]
 ===== Decision Tree =====
Media 0.785248447204969
[0.80434783 0.83695652 0.7826087  0.81521739 0.83695652 0.7826087
 0.79347826 0.77173913 0.7032967  0.72527473]
 ===== KNN =====
Media 0.7035236502627807
[0.65217391 0.69565217 0.85869565 0.80434783 0.72826087 0.60869565
 0.7826087  0.66304348 0.61538462 0.62637363]


4. Scalare i valori di RestingBP, Cholesterol e MaxHR in un intervallo tra 0 e 1 utilizzando la funzione MinMAxScaler.

In [30]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
columns_to_scale = ['RestingBP', 'Cholesterol', 'MaxHR']
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

In [31]:
df

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,1,1,0.70,0.479270,0,1,0.788732,0,0.0,2,0
1,49,0,2,0.80,0.298507,0,1,0.676056,0,1.0,1,1
2,37,1,1,0.65,0.469320,0,2,0.267606,0,0.0,2,0
3,48,0,0,0.69,0.354892,0,1,0.338028,1,1.5,1,1
4,54,1,2,0.75,0.323383,0,1,0.436620,0,0.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,1,3,0.55,0.437811,0,1,0.507042,0,1.2,1,1
914,68,1,0,0.72,0.320066,1,1,0.570423,0,3.4,1,1
915,57,1,0,0.65,0.217247,0,1,0.387324,1,1.2,1,1
916,57,0,1,0.65,0.391376,0,0,0.802817,0,0.0,1,1


In [33]:
X_train, X_test, Y_train, Y_test = train_test_split(df.drop('HeartDisease', axis=1), df['HeartDisease'], test_size=0.25, stratify=df['HeartDisease'], random_state=42)

decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(X_train, Y_train)
Y_pred = decision_tree_model.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
print("Accuratezza del modello Decision Tree:", accuracy)

Accuratezza del modello Decision Tree con le feature più correlate: 0.8


5. Analizzare la correlazione tra le feature del dataset, creare un dataframe che contiene, oltre alla colonna target, le 5
feature più correlate (positivamente) al target. La predizione effettuata con DecisionTree migliora? si calcola con dataframe.core->matrice di correlazione che fa vedere ogni feature quanto è correlata, mi interessa vedere quanto il target è correlato con le altre feature. posso vedere le fetaure più correlate e posso vedere per es prendendo tutte le 5 feature più correlate e togliendo le altre valuto se così predizione migliora o peggiora.

In [19]:
correlation_matrix = df.corr() #Calcola la correlazione tra tutte le colonne numeriche del DataFrame df
#Seleziona le 5 feature più correlate con il target (HeartDisease)
top_correlated_features = correlation_matrix['HeartDisease'].sort_values(ascending=False)[0:6].index.tolist()
new_df = df[['HeartDisease'] + top_correlated_features] #unisce due liste: una lista che contiene solo la colonna target (['HeartDisease']) e la lista delle feature più correlate (top_correlated_features)
#di base creo un nuovo dataframe contenente solo le feature più correlate e il target
#faccio il training e testo su questo
X_train, X_test, Y_train, Y_test = train_test_split(new_df.drop('HeartDisease', axis=1), new_df['HeartDisease'], test_size=0.25, stratify=new_df['HeartDisease'], random_state=42)

decision_tree_model = DecisionTreeClassifier()
decision_tree_model.fit(X_train, Y_train)
Y_pred = decision_tree_model.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
print("Accuratezza del modello Decision Tree con le feature più correlate:", accuracy)

Accuratezza del modello Decision Tree con le feature più correlate: 0.7130434782608696


6. A partire dal dataset iniziale (in cui sono stati eliminati eventuali attributi inutili ed eventuali istanze con valori nulli e sono
stati opportunamente trasformati i valori categorici) trovare i valori migliori dei parametri criterion e max_depth del
classificatore DecisionTree. Come varia l’accuratezza della predizione?

valuto una serie di valori epr questi 2 iperparametri del modello decision three e vedo combinazione migliore

In [35]:
from sklearn.model_selection import GridSearchCV

#griglia di ricerca degli iperparametri ad testare:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 5, 10, 15, 20]
}

decision_tree_model = DecisionTreeClassifier()
#Imposta la ricerca a griglia
grid_search = GridSearchCV(estimator=decision_tree_model, param_grid=param_grid, cv=5, scoring='accuracy')

new_dataset = pd.read_csv("heart.csv")
new_dataset = new_dataset.dropna()

le = LabelEncoder()
for column in new_dataset.columns:
    if new_dataset[column].dtype == object:
        new_dataset[column] = le.fit_transform(new_dataset[column])

X_train, X_test, Y_train, Y_test = train_test_split(new_dataset.drop('HeartDisease', axis=1), new_dataset['HeartDisease'], test_size=0.25, stratify=new_dataset['HeartDisease'], random_state=42)

grid_search.fit(X_train, Y_train)
print("Migliori parametri:", grid_search.best_params_)

best_decision_tree_model = grid_search.best_estimator_
Y_pred = best_decision_tree_model.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
print("Accuratezza del modello Decision Tree con i migliori parametri:", accuracy)

Migliori parametri: {'criterion': 'entropy', 'max_depth': 5}
Accuratezza del modello Decision Tree con i migliori parametri: 0.8347826086956521
